In [65]:
import json
import ast

from pyspark.mllib.linalg.distributed import RowMatrix

from pyspark.sql import Row
from pyspark.sql import SparkSession 

from pyspark.mllib.linalg import Vectors

import tqdm

from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import SparseVector

from scipy.sparse import find

import matplotlib.pyplot as plt
import numpy as np

from collections import Counter

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

import random

NUMCLASTERS = 8

In [2]:
spark = SparkSession \
       .builder \
       .master('yarn') \
       .enableHiveSupport() \
       .getOrCreate()

#.master('yarn') \

In [3]:
df_lst = spark.read.table('prod.announcement_lst').repartition(100)
#df_lst.printSchema()

In [4]:
import math
from numpy import array
DICT = {'bedRent': 1,
        'commercialLandRent': 2,
        'dailyHouseRent': 3,
        'flatRent': 4,
        'flatSale': 5,
        'freeAppointmentObjectRent': 6,
        'officeRent': 7,
        'roomRent': 8,
        'shoppingAreaRent': 9,
        'warehouseSale': 10,
        'cottageSale': 11,
        'flatShareSale': 12,
        'freeAppointmentObjectSale': 13,
        'garageRent': 14,
        'garageSale': 15,
        'industryRent': 16,
        'newBuildingFlatSale': 17,
        'townhouseRent': 18,
        'buildingSale': 19,
        'cottageRent': 20,
        'dailyFlatRent': 21,
        'houseRent': 22,
        'houseSale': 23,
        'landSale': 24,
        'roomSale': 25,
        'warehouseRent': 26,
        'buildingRent': 27,
        'shoppingAreaSale': 28,
        'townhouseSale': 29,
        'businessRent': 30,
        'businessSale': 31,
        'dailyBedRent': 32,
        'dailyRoomRent': 33,
        'houseShareRent': 34,
        'houseShareSale': 35,
        'industrySale': 36,
        'officeSale': 37,
        'commercialLandSale': 38}

In [5]:
def lambda1(x):
    if x['priceType'] == 'squareMeter':
        if (x['price'] is None) or (x['totalarea'] is None):
            totalprice = 0
        else:
            totalprice = x['price'] * x['totalarea']
    elif x['price'] is None:
        totalprice = 0
    else: 
        totalprice = x['price']
    
    if x['floornumber'] is None:
        floornumber = 0
    else:
        floornumber = x['floornumber']
    
    if x['floorscount'] is None:
        floorscount = 0
    else:
        floorscount = x['floorscount']
    
    if x['category'] is None:
        category = 0
    else:
        category = DICT[x['category']] * 2
    
    if x['roomscount'] is None:
        roomscount = 0
    else:
        roomscount = x['roomscount']
    
    if x['totalarea'] is None:
        totalarea = 0
    else:
        totalarea = x['totalarea']
    
    return np.array([x['announcementid'],
                     floornumber, 
                     floorscount, 
                     category, 
                     roomscount, 
                     totalarea, 
                     totalprice])

In [6]:
rddSoprWithId = df_lst.select('announcementid', 
                         'floornumber', 
                         'floorscount', 
                         'category', 
                         'roomscount', 
                         'totalarea', 
                         'price',
                         'priceType') \
                 .dropDuplicates() \
                 .rdd \
                 .map(lambda x: lambda1(x)) #.show()
X = rddSoprWithId.map(lambda x: x[1:]).randomSplit([(2918553 - 1000)/2918553, 1000/2918553])[1]
#rdd_lst = df_lst.limit(10000).rdd#.show()

In [7]:
model = KMeans.train(X,
                     NUMCLASTERS, 
                     initializationMode="k-means||", 
                     initializationSteps=5, 
                     epsilon=1e-4)

#model.save("target/org/apache/spark/PythonKMeansExample/KMeansModel")

In [8]:
def addpredict(x):
    p = model.predict(x[1:])
    return (int(x[0]), p)

In [9]:
rddSoprWithIdPred = rddSoprWithId.map(lambda x: addpredict(x))

In [10]:
DICT_W_FOR_PAGE_TYPE = {"Card" : 3,
                        "CardJK" : 2,
                        "Listing" : 1,
                        "ListingFavorites" : 5}

DICT_W_FOR_EVENT_TYPE = {"card_show" : 3,
                        "phone_show" : 10}


data = [Row(page_type='Card', event_type='card_show', value = DICT_W_FOR_PAGE_TYPE["Card"] * DICT_W_FOR_EVENT_TYPE["card_show"]), 
        Row(page_type='CardJK', event_type='card_show', value = DICT_W_FOR_PAGE_TYPE["CardJK"] * DICT_W_FOR_EVENT_TYPE["card_show"]), 
        Row(page_type='Listing', event_type='card_show', value = DICT_W_FOR_PAGE_TYPE["Listing"] * DICT_W_FOR_EVENT_TYPE["card_show"]), 
        Row(page_type='ListingFavorites', event_type='card_show', value = DICT_W_FOR_PAGE_TYPE["ListingFavorites"] * DICT_W_FOR_EVENT_TYPE["card_show"]), 
        Row(page_type='Card', event_type='phone_show', value = DICT_W_FOR_PAGE_TYPE["Card"] * DICT_W_FOR_EVENT_TYPE["phone_show"]), 
        Row(page_type='CardJK', event_type='phone_show', value = DICT_W_FOR_PAGE_TYPE["CardJK"] * DICT_W_FOR_EVENT_TYPE["phone_show"]), 
        Row(page_type='Listing', event_type='phone_show', value = DICT_W_FOR_PAGE_TYPE["Listing"] * DICT_W_FOR_EVENT_TYPE["phone_show"]), 
        Row(page_type='ListingFavorites', event_type='phone_show', value = DICT_W_FOR_PAGE_TYPE["ListingFavorites"] * DICT_W_FOR_EVENT_TYPE["phone_show"])] 

dfdict = spark.createDataFrame(sc.parallelize(data))
dfdict.createOrReplaceTempView("dfdict")

In [11]:
user_item = spark.sql("""select a.user_id, a.offer_id from 
                            (select user_id, offer_id
                                from prod.mles_sopr
                                where ptn_dadd between '2019-06-01' and '2019-06-21') as a
                        inner join 
                            (select user_id, offer_id
                                from prod.mles_sopr
                                where ptn_dadd between '2019-06-22' and '2019-07-03') as b
                        on a.user_id = b.user_id 
                        where a.user_id != 'noid'
                        limit 10000
                  """).repartition(100).createOrReplaceTempView("user_item")

In [14]:
sqlDF = spark.sql("""select distinct a.offer_num, a.offer_id, b.user_num, b.user_id, c.value, ptn_dadd
                    from (select user_id, offer_id, value, ptn_dadd
                            from dfdict as a, prod.mles_sopr as b 
                            where a.page_type = b.page_type and a.event_type = b.event_type) as c
                         INNER JOIN (select row_number() OVER (ORDER BY a.user_id) as user_num, a.user_id 
                            from (select distinct user_id from user_item) as a) as b on c.user_id = b.user_id
                         INNER JOIN (select row_number() OVER (ORDER BY a.offer_id) as offer_num, a.offer_id 
                            from (select distinct offer_id from user_item) as a) as a on c.offer_id = a.offer_id 
                  """).repartition(100)

In [15]:
sqlDFtrain = sqlDF.where("ptn_dadd between '2019-06-01' and '2019-06-21'")

In [16]:
sqlDFtest = sqlDF.where("ptn_dadd between '2019-06-22' and '2019-07-03'")

In [20]:
rddtrain = sqlDFtrain.rdd.map(lambda x: (x['offer_id'], x)).join(rddSoprWithIdPred)

In [58]:
listrdd = []
for i in range(NUMCLASTERS):
    test = rddtrain.filter(lambda x: x[1][1] == i)
    if test.isEmpty():
        listrdd.append(None)
        continue
    else:
        listrdd.append(spark.createDataFrame(test.map(lambda x: x[1][0])))
        continue

In [69]:
def PoolALS(x):
    als = ALS(maxIter=5, regParam=0.01, userCol="user_num", itemCol="offer_num", ratingCol="value",
          coldStartStrategy="drop")
    model = als.fit(x)
    return model

In [79]:
k = 10
modelList = []
recsList = []
with tqdm.tqdm(total=NUMCLASTERS) as progress:
    for rdds in listrdd:
        if rdds:
            model = PoolALS(rdds)
            modelList.append(model)
            user_recs = model.recommendForAllUsers(k)
            recsList.append(user_recs)
        else:
            modelList.append(None)
            recsList.append(None)
        progress.update(1)

100%|██████████| 8/8 [01:09<00:00, 12.34s/it]


In [73]:
num_users = spark.sql("""select count(*) from (select distinct user_id from user_item)
                  """).collect()[0]['count(1)']

In [ ]:
sum = [0] * NUMCLASTERS
with tqdm.tqdm(total=20) as progress:
    for j in range(20):
        progress.update(1)
        user = int(random.random() * num_users)
        t = sqlDFtest.where("user_num = " + str(user)).select("offer_num").collect()
        viewed = [i['offer_num'] for i in t]
        if len(viewed) == 0:
            continue
        
        for l in range(NUMCLASTERS):
            recs = recsList[l]
            if recs:
                t = recs.where('user_num = ' + str(user)).collect()
                if len(t) == 0:
                    continue

                t = t[0]['recommendations']
                recom = [i['offer_num'] for i in t]


                sum[l] += len(np.intersect1d(recom, viewed))/k
        #print("    ", user, " sum =", sum)


 95%|█████████▌| 19/20 [06:48<00:23, 23.73s/it]